In [ ]:
# VEF ACADEMY - MACHINE LEARNING – 2021 
# Solution for Problem 2: Implementation of a Convolutional Neural Network
# Written by Hieu Pham

# Import required packages and libs 
import numpy as np
import torch
import torchvision

# Set hyperparameter for training 
n_epochs = 30
batch_size_train = 128
batch_size_test = 1000
learning_rate = 0.01

# Seeding 
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

# Preparing the dataset

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)


# 2(a) Reshape the data with dimensions of the form samples×1×28×28.

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_data.shape)

# Building the CNN network 

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.dropout = nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


model = Net()

# Using thepytorch-summarypackage to verify your implementation
from torchsummary import summary
summary(model, (1, 28, 28))


# 2(c) Train  your  CNN  model  for  30  epochs  with  cross-entropy loss function, 
# SGD optimizer, and batch size of 128. Report the accuracy on the test set.

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

test()  
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()



torch.Size([1000, 1, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
            Linear-3                  [-1, 128]       1,179,776
           Dropout-4                  [-1, 128]               0
            Linear-5                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.45
Params size (MB): 4.58
Estimated Total Size (MB): 5.03
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3065, Accuracy: 1468/10000 (15%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313666
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.198941
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.097548
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.920699
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.689883
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.399448
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.018384
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.909708
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.666779
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.689804
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.629457
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.553306
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.389417
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.546263
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.616015
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.435132
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.501023
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.378801
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.591958
Train Ep